# Bakery Inventory
_This notebook was created by Oluwaferanmi Alalade on the 10th of September,2022_

### Importing the necessary libraries
The libraries used in this notebook were pandas, which is used for data manipulation and analysis, and datetime, which is used in dealing with dates

In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
df = pd.read_csv(r"C:\Users\Oluwaferanmi\Documents\settlements_data_JS.csv") #Reading the csv file into this notebook

In [3]:
df.head() #Display the first five rows of the data

,Order ID,Activity Date,Bakery Item,Quantity,Dee's Location,Activity Type,Inventory Date
0,344545,21/08/2020,CCC,170,1JS,Order,NaN
1,4384815,20/08/2020,CCC,80,2WS,Order,NaN
2,24084,23/08/2020,AAPL,250,2WS,Shipment,NaN
3,3092445,21/08/2020,BBRD,50,2WS,Order,NaN
4,2373263,21/08/2020,BBRD,120,1JS,Order,NaN


In [4]:
df["Activity Date"].value_counts() #Display the unique activity dates and the count of each date

21/08/2020    67
22/08/2020    66
20/08/2020    60
23/08/2020     9
Name: Activity Date, dtype: int64

The whole dataset is for a 4 day period

### InventoryDate function
This function takes in the Activity date, and returns the inventory date, the bakery has three outlets:
 - 1JS: located on Jane Street, NY: Shipping to this location takes 1 day
 - 2WS: located on Wall Street,NY: Shipping also takes 1 day
 - 3PUFF: located in Poughkeepsie: Shipping takes two days
 
When an order is placed, it takes two days before the products are ready for pickup

In [5]:
def InventoryDate(data):
    Invent_Date = []#Inventory Date
    for index, record in data.iterrows():
        if record["Activity Type"] == "Order":
            date = datetime.strptime(record["Activity Date"], "%d/%m/%Y")
            record["Inventory Date"] = date + timedelta(days=2)#An order takes two days to be completed
        elif record["Activity Type"] == "Shipment":
            if record["Dee's Location"] == "1JS" or record["Dee's Location"] == "2WS":
                date = datetime.strptime(record["Activity Date"], "%d/%m/%Y")
                record["Inventory Date"] = date + timedelta(days=1) #Add 1 day to when the product leaves the mega bakery
            elif record["Dee's Location"] == "3PUFF":
                date = datetime.strptime(record["Activity Date"], "%d/%m/%Y")
                record["Inventory Date"] = date + timedelta(days=2) #Add 2 days
        else:
            continue

        Invent_Date.append(record["Inventory Date"]) #Append the inventory date to the empty list
    return Invent_Date
    

In [6]:
df["Inventory Date"] = InventoryDate(df) #Cast the inventory dates into the original dataframe

In [7]:
df.head() #Display the first five columns

,Order ID,Activity Date,Bakery Item,Quantity,Dee's Location,Activity Type,Inventory Date
0,344545,21/08/2020,CCC,170,1JS,Order,2020-08-23
1,4384815,20/08/2020,CCC,80,2WS,Order,2020-08-22
2,24084,23/08/2020,AAPL,250,2WS,Shipment,2020-08-24
3,3092445,21/08/2020,BBRD,50,2WS,Order,2020-08-23
4,2373263,21/08/2020,BBRD,120,1JS,Order,2020-08-23


In [8]:
shipment_df = df.loc[df["Activity Type"] == "Shipment"] #Locate the shipment activities
order_df = df.loc[df["Activity Type"] == "Order"]#Locate the order activities

## Shipment

In [9]:
shipment_df.groupby(["Bakery Item"])[["Quantity"]].sum() #Group the shipments by products and display the quantity

,Quantity
Bakery Item,
AAPL,2100
BBRD,6000
CCC,4800


From the above, it is seen that in the four day period, 2100 Applepies, 6000 banana bread and 4800 Chocolate chip cookies were shipped in the four day period

## Order 

In [10]:
order_df.groupby("Bakery Item").sum()[["Quantity"]] #Group the orders by products and display the quantity

,Quantity
Bakery Item,
AAPL,3960
BBRD,6420
CCC,5710


3960 apple pies were ordered, 6420 banana bread and 5710 chocolate chip cookies 

## Shipping to different locations
### Finding the quantity of each product shipped to the different locations on the 22nd, 23rd and 24th

#### The inventory dates

In [11]:
shipment_df["Inventory Date"].value_counts()

2020-08-24    9
2020-08-23    9
2020-08-22    6
2020-08-25    3
Name: Inventory Date, dtype: int64

In [12]:
#Grouping the different shipments by the inventory dates
d = shipment_df.groupby("Inventory Date")

### First Date - 22/08/2020

In [13]:
#Getting the information on the first inventory date which is the 22nd of august, 2020

first_date = d.get_group("2020-08-22")
first_date

,Order ID,Activity Date,Bakery Item,Quantity,Dee's Location,Activity Type,Inventory Date
8,4755979,21/08/2020,AAPL,150,2WS,Shipment,2020-08-22
22,2793436,21/08/2020,BBRD,900,1JS,Shipment,2020-08-22
29,3836973,21/08/2020,BBRD,700,2WS,Shipment,2020-08-22
90,2899092,21/08/2020,CCC,450,2WS,Shipment,2020-08-22
102,4546348,21/08/2020,CCC,450,1JS,Shipment,2020-08-22
197,131584,21/08/2020,AAPL,350,1JS,Shipment,2020-08-22


Analyzing information on the quantity of products shipped to the different locations on the first inventory date
 - js_first_ship being the dataframe which shows the quantity of each bakery items shipped to 1JS outlet on the first inventory date
 - ws_first_ship being the dataframe which shows the quantity of each bakery items shipped to 2WS outlet on the first inventory date

In [14]:
location_first = first_date.groupby("Dee's Location")#Grouping by the different outlet locations for the first inv date
js_first_ship = location_first.get_group("1JS")[["Bakery Item","Quantity"]]
js_first_ship = js_first_ship.groupby("Bakery Item").sum()["Quantity"]#Group by Item and display quantity

ws_first_ship = location_first.get_group("2WS")[["Bakery Item","Quantity"]]
ws_first_ship = ws_first_ship.groupby("Bakery Item").sum()["Quantity"]#Group by Item and display quantity

In [15]:
print(f"Shipment from 1JS on the second date: \n{js_first_ship}")
print(f"Shipment from 2WS on the second date: \n{ws_first_ship}")

Shipment from 1JS on the second date: 
Bakery Item
AAPL    350
BBRD    900
CCC     450
Name: Quantity, dtype: int64
Shipment from 2WS on the second date: 
Bakery Item
AAPL    150
BBRD    700
CCC     450
Name: Quantity, dtype: int64


In [16]:
try:
    pf_first_ship = location_first.get_group("3PUFF")[["Bakery Item","Quantity"]]
    pf_first_ship = pf_first_ship.groupby("Bakery Item").sum()["Quantity"]#Group by Item and display quantity
    print(pf_first_ship)
except Exception as e:
    print(e)
    print("There were no shipments to this location on this date")

'3PUFF'
There were no shipments to this location on this date


The error above shows no shipment landed in poughkeepsie on the 22nd of August, 2020

### Second date - 23/08/2020

In [17]:
second_date = d.get_group("2020-08-23")#get information on the 23rd
second_date

,Order ID,Activity Date,Bakery Item,Quantity,Dee's Location,Activity Type,Inventory Date
19,199073,21/08/2020,AAPL,200,3PUFF,Shipment,2020-08-23
82,905782,22/08/2020,AAPL,350,2WS,Shipment,2020-08-23
97,4618025,22/08/2020,CCC,450,1JS,Shipment,2020-08-23
118,1452264,22/08/2020,CCC,350,2WS,Shipment,2020-08-23
127,2724679,21/08/2020,BBRD,400,3PUFF,Shipment,2020-08-23
136,368117,22/08/2020,BBRD,600,2WS,Shipment,2020-08-23
164,2074756,22/08/2020,BBRD,1000,1JS,Shipment,2020-08-23
177,4928620,22/08/2020,AAPL,150,1JS,Shipment,2020-08-23
183,163422,21/08/2020,CCC,700,3PUFF,Shipment,2020-08-23


 Second date - 23rd of august 2020
 - js_sec_ship being the dataframe which shows the quantity of each bakery items shipped to 1JS outlet on the second inventory date
 - ws_sec_ship being the dataframe which shows the quantity of each bakery items shipped to 2WS outlet on the second inventory date
 - pf_sec_ship being the dataframe which shows the quantity of each bakery items shipped to 2WS outlet on the second inventory date

In [18]:
location_sec = second_date.groupby("Dee's Location")
js_sec_ship = location_sec.get_group("1JS")[["Bakery Item","Quantity"]]#Get Item and quantity info to 1JS
js_sec_ship = js_sec_ship.groupby("Bakery Item").sum()["Quantity"]#Group by Item and display quantity

ws_sec_ship = location_sec.get_group("2WS")[["Bakery Item","Quantity"]]
ws_sec_ship = ws_sec_ship.groupby("Bakery Item").sum()["Quantity"]#Group by Item and display quantity

pf_sec_ship = location_sec.get_group("3PUFF")[["Bakery Item","Quantity"]]
pf_sec_ship = pf_sec_ship.groupby("Bakery Item").sum()["Quantity"]#Group by Item and display quantity

In [19]:
print(f"Shipment from 1JS on the second date: \n{js_sec_ship}")
print(f"Shipment from 2WS on the second date: \n{ws_sec_ship}")
print(f"Shipment from 3PUFF on the second date: \n{pf_sec_ship}")

Shipment from 1JS on the second date: 
Bakery Item
AAPL     150
BBRD    1000
CCC      450
Name: Quantity, dtype: int64
Shipment from 2WS on the second date: 
Bakery Item
AAPL    350
BBRD    600
CCC     350
Name: Quantity, dtype: int64
Shipment from 3PUFF on the second date: 
Bakery Item
AAPL    200
BBRD    400
CCC     700
Name: Quantity, dtype: int64


### Third date - 24th August 2020

In [20]:
third_date = d.get_group("2020-08-24")#Get more information on the 24th
third_date

,Order ID,Activity Date,Bakery Item,Quantity,Dee's Location,Activity Type,Inventory Date
2,24084,23/08/2020,AAPL,250,2WS,Shipment,2020-08-24
21,612627,23/08/2020,AAPL,300,1JS,Shipment,2020-08-24
38,4626866,22/08/2020,CCC,800,3PUFF,Shipment,2020-08-24
41,1088540,23/08/2020,BBRD,800,1JS,Shipment,2020-08-24
52,1473291,23/08/2020,CCC,450,1JS,Shipment,2020-08-24
69,1901900,23/08/2020,CCC,550,2WS,Shipment,2020-08-24
91,2683759,22/08/2020,BBRD,400,3PUFF,Shipment,2020-08-24
120,3059395,23/08/2020,BBRD,800,2WS,Shipment,2020-08-24
160,580731,22/08/2020,AAPL,200,3PUFF,Shipment,2020-08-24


In [21]:
location_third = third_date.groupby("Dee's Location")
js_third_ship = location_third.get_group("1JS")[["Bakery Item","Quantity"]]#Get Item and quantity info to 1JS
js_third_ship = js_third_ship.groupby("Bakery Item").sum()["Quantity"]#Group by Item and display quantity

ws_third_ship = location_third.get_group("2WS")[["Bakery Item","Quantity"]]#Get Item and quantity info to 2WS
ws_third_ship = ws_third_ship.groupby("Bakery Item").sum()["Quantity"]#Group by Item and display quantity

pf_third_ship = location_third.get_group("3PUFF")[["Bakery Item","Quantity"]]#Get Item and quantity info to 3PUFF
pf_third_ship = pf_third_ship.groupby("Bakery Item").sum()["Quantity"]#Group by Item and display quantity

In [22]:
print(f"Shipment from 1JS on the third date: \n{js_third_ship}")
print(f"Shipment from 2WS on the third date: \n{ws_third_ship}")
print(f"Shipment from 3PUFF on the third date: \n{ws_third_ship}")

Shipment from 1JS on the third date: 
Bakery Item
AAPL    300
BBRD    800
CCC     450
Name: Quantity, dtype: int64
Shipment from 2WS on the third date: 
Bakery Item
AAPL    250
BBRD    800
CCC     550
Name: Quantity, dtype: int64
Shipment from 3PUFF on the third date: 
Bakery Item
AAPL    250
BBRD    800
CCC     550
Name: Quantity, dtype: int64


## Ordersfrom different locations
### Finding the quantity of each product ordered from the different locations on the 22nd, 23rd and 24th

In [23]:
e = order_df.groupby("Inventory Date") #Grouping the different orders by the inventory dates

### First Inventory Date

In [24]:
first_ord = e.get_group("2020-08-22")

In [25]:
location_first = first_ord.groupby("Dee's Location")

js_first_ord = location_first.get_group("1JS")#Get Item and quantity info to 1JS
js_first_ord = js_first_ord.groupby("Bakery Item").sum()["Quantity"]

ws_first_ord = location_first.get_group("2WS")#Get Item and quantity info to 2WS
ws_first_ord = ws_first_ord.groupby("Bakery Item").sum()["Quantity"]

pf_first_ord = location_first.get_group("3PUFF")#Get Item and quantity info to 3PUFF
pf_first_ord = pf_first_ord.groupby("Bakery Item").sum()["Quantity"]#Group by Item and display quantity

In [26]:
print(f"Order from 1JS on the first date: \n{js_first_ord}")
print(f"Order from 2WS on the first date: \n{ws_first_ord}")
print(f"Order from 3PUFF on the first date: \n{pf_first_ord}")

Order from 1JS on the first date: 
Bakery Item
AAPL    680
BBRD    520
CCC     440
Name: Quantity, dtype: int64
Order from 2WS on the first date: 
Bakery Item
AAPL    320
BBRD    530
CCC     500
Name: Quantity, dtype: int64
Order from 3PUFF on the first date: 
Bakery Item
AAPL    420
BBRD    250
CCC     290
Name: Quantity, dtype: int64


### Second Inventory Date

In [27]:
sec_ord = e.get_group("2020-08-23")

In [28]:
location_sec_ord = sec_ord.groupby("Dee's Location")

js_sec_ord = location_sec_ord.get_group("1JS")#Get Item and quantity info to 1JS
js_sec_ord = js_sec_ord.groupby("Bakery Item").sum()["Quantity"]

ws_sec_ord = location_sec_ord.get_group("2WS")#Get Item and quantity info to 2WS
ws_sec_ord = ws_sec_ord.groupby("Bakery Item").sum()["Quantity"]

pf_sec_ord = location_sec_ord.get_group("3PUFF")#Get Item and quantity info to 3PUFF
pf_sec_ord = pf_sec_ord.groupby("Bakery Item").sum()["Quantity"]#Group by Item and display quantity

In [29]:
print(f"Order from 1JS on the second date: \n{js_sec_ord}")
print(f"Order from 2WS on the second date: \n{ws_sec_ord}")
print(f"Order from 3PUFF on the second date: \n{pf_sec_ord}")

Order from 1JS on the second date: 
Bakery Item
AAPL     520
BBRD    1190
CCC     1710
Name: Quantity, dtype: int64
Order from 2WS on the second date: 
Bakery Item
AAPL    330
BBRD    660
CCC     230
Name: Quantity, dtype: int64
Order from 3PUFF on the second date: 
Bakery Item
AAPL    410
BBRD    600
CCC     420
Name: Quantity, dtype: int64


### Third Inventory Date

In [30]:
third_ord = e.get_group("2020-08-24")

In [31]:
location_third_ord = third_ord.groupby("Dee's Location")

js_third_ord = location_third_ord.get_group("1JS")#Get Item and quantity info to 1JS
js_third_ord = js_third_ord.groupby("Bakery Item").sum()["Quantity"]

ws_third_ord = location_third_ord.get_group("2WS")#Get Item and quantity info to 2WS
ws_third_ord = ws_third_ord.groupby("Bakery Item").sum()["Quantity"]

pf_third_ord = location_third_ord.get_group("3PUFF")#Get Item and quantity info to 3PUFF
pf_third_ord = pf_third_ord.groupby("Bakery Item").sum()["Quantity"]#Group by Item and display quantity

In [32]:
print(f"Order from 1JS on the third date: \n{js_sec_ord}")
print(f"Order from 2WS on the third date: \n{ws_sec_ord}")
print(f"Order from 3PUFF on the third date: \n{pf_sec_ord}")

Order from 1JS on the third date: 
Bakery Item
AAPL     520
BBRD    1190
CCC     1710
Name: Quantity, dtype: int64
Order from 2WS on the third date: 
Bakery Item
AAPL    330
BBRD    660
CCC     230
Name: Quantity, dtype: int64
Order from 3PUFF on the third date: 
Bakery Item
AAPL    410
BBRD    600
CCC     420
Name: Quantity, dtype: int64


## To calculate the quantity of each product in the inventory of each outlet and the end of each day
The earliest shipment to any of the outlet stores landed on the 22nd of August, before then it is assumed that all the outlets had 500 of each product in their inventory, and it is stated that AAPL shipment was on hold due to logistic issues which means the quantity of Apple pies in each of the outlets remained 500 throughout the period

## Jane Street Location
The quantity of each product left in the inventory of the outlet on Jane Street on the 3 dates was calculatedin this section.

In [33]:
js_first = {}

for index,(js1_ship, js1_ord) in enumerate(zip(js_first_ship.values, js_first_ord.values)):
    if index == 0:
        js_first["AAPL"] = [js1_ship, js1_ord]
    elif index == 1:
        js_first["BBRD"] = [js1_ship, js1_ord]
    elif index == 2:
        js_first["CCC"] = [js1_ship, js1_ord]
        
js_second = {}

for index,(js2_ship, js2_ord) in enumerate(zip(js_sec_ship.values, js_sec_ord.values)):
    if index == 0:
        js_second["AAPL"] = [js2_ship, js2_ord]
    elif index == 1:
        js_second["BBRD"] = [js2_ship, js2_ord]
    elif index == 2:
        js_second["CCC"] = [js2_ship, js2_ord]
        
js_third = {}

for index,(js3_ship, js3_ord) in enumerate(zip(js_third_ship.values, js_third_ord.values)):
    if index == 0:
        js_third["AAPL"] = [js3_ship, js3_ord]
    elif index == 1:
        js_third["BBRD"] = [js3_ship, js3_ord]
    elif index == 2:
        js_third["CCC"] = [js3_ship, js3_ord]

In [34]:
print(f"First day shipment and orders: {js_first}")
print(f"Second day shipment and orders: {js_second}")
print(f"Third day shipment and orders: {js_third}")

First day shipment and orders: {'AAPL': [350, 680], 'BBRD': [900, 520], 'CCC': [450, 440]}
Second day shipment and orders: {'AAPL': [150, 520], 'BBRD': [1000, 1190], 'CCC': [450, 1710]}
Third day shipment and orders: {'AAPL': [300, 770], 'BBRD': [800, 1310], 'CCC': [450, 1090]}


In [35]:
js = {} #This dictionary will later contain the order and expected shipment quantities on the three dates
        #at jane street

js["First Day"] = js_first
js["Second Day"] = js_second
js["Third Day"] = js_third

In [36]:
js

{'First Day': {'AAPL': [350, 680], 'BBRD': [900, 520], 'CCC': [450, 440]},
 'Second Day': {'AAPL': [150, 520], 'BBRD': [1000, 1190], 'CCC': [450, 1710]},
 'Third Day': {'AAPL': [300, 770], 'BBRD': [800, 1310], 'CCC': [450, 1090]}}

In [37]:
def ord_ship_bal(data):
    """
    This function returns
    the difference between
    two numbers,
    In this case, the difference
    in shipment and order for
    each day
    """
    balance = int(data[0]) - int(data[1])
    return balance

## Inventory at the end of the three days - Jane Street
 - 22/08/2020
 - 23/08/2020
 - 24/08/2020
 
 At the beginning of the time period, we assumed each location had in their inventory 500 of each product. Also due to some logistic issues, there would not be any shipment of apple pies to any of the locations.

In [38]:
base_aapl = 500 #Inventory quantity at the beginning of the time period
base_bbrd = 500
base_ccc = 500
invent_day = {}
day1_js = {}
day2_js = {}
day3_js = {}

for index, day in enumerate(js.values()):    
    if index == 0: #First day - 22nd
        day1_js["AAPL"] = base_aapl - day.get("AAPL")[1] #get the order quantity of appe pies 
        day1_js["BBRD"] = base_bbrd + ord_ship_bal(day.get("BBRD"))#get the quantity of BBRD left after the first day
        day1_js["CCC"] = base_ccc + ord_ship_bal(day.get("CCC"))#get the quantity of CCC left after the first day
        
        #The base quantities will then be updated to the quantity left after the first day
        base_aapl = day1_js.get("AAPL")#Inventory quantities at the beginning of the second day
        base_bbrd = day1_js["BBRD"]
        base_ccc = day1_js["CCC"]
    
    if index == 1:#Second day - 23rd
        #Quantities left in the inventory at the end of the second day
        day2_js["AAPL"] = base_aapl - day.get("AAPL")[1]
        day2_js["BBRD"] = base_bbrd + ord_ship_bal(day.get("BBRD"))
        day2_js["CCC"] = base_ccc + ord_ship_bal(day.get("CCC"))
        
        base_aapl = day2_js.get("AAPL")#Inventory quantities at the beginning of the third day
        base_bbrd = day2_js["BBRD"]
        base_ccc = day2_js["CCC"]
    
    if index == 2:#Third day - 24th
        day3_js["AAPL"] = base_aapl - day.get("AAPL")[1]
        day3_js["BBRD"] = base_bbrd + ord_ship_bal(day.get("BBRD"))#get the quantity of BBRD left after the first day
        day3_js["CCC"] = base_ccc + ord_ship_bal(day.get("CCC"))#get the quantity of CCC left after the first day

In [39]:
day1_js

{'AAPL': -180, 'BBRD': 880, 'CCC': 510}

In [40]:
day2_js

{'AAPL': -700, 'BBRD': 690, 'CCC': -750}

In [41]:
day3_js

{'AAPL': -1470, 'BBRD': 180, 'CCC': -1390}

In [42]:
js_invent = {}

js_invent["22/08/2020 inventory"] = day1_js
js_invent["23/08/2020 inventory"] = day2_js
js_invent["24/08/2020 inventory"] = day3_js

In [43]:
js_invent

{'22/08/2020 inventory': {'AAPL': -180, 'BBRD': 880, 'CCC': 510},
 '23/08/2020 inventory': {'AAPL': -700, 'BBRD': 690, 'CCC': -750},
 '24/08/2020 inventory': {'AAPL': -1470, 'BBRD': 180, 'CCC': -1390}}

## Wall Street Location
The quantity of each product left in the inventory of the outlet on Wall Street on the 3 dates was calculated in this section.

In [44]:
ws_first = {}

for index,(ws1_ship, ws1_ord) in enumerate(zip(ws_first_ship.values, ws_first_ord.values)):
    if index == 0:
        ws_first["AAPL"] = [ws1_ship, ws1_ord]
    elif index == 1:
        ws_first["BBRD"] = [ws1_ship, ws1_ord]
    elif index == 2:
        ws_first["CCC"] = [ws1_ship, ws1_ord]
        
ws_second = {}#The order and expected shipment(The shipment wasnt processed) quantities on the second day

for index,(ws2_ship, ws2_ord) in enumerate(zip(ws_sec_ship.values, ws_sec_ord.values)):
    if index == 0:
        ws_second["AAPL"] = [ws2_ship, ws2_ord]
    elif index == 1:
        ws_second["BBRD"] = [ws2_ship, ws2_ord]
    elif index == 2:
        ws_second["CCC"] = [ws2_ship, ws2_ord]
        
ws_third = {} #The order and expected shipment(The shipment wasnt processed) quantities on the third day

for index,(ws3_ship, ws3_ord) in enumerate(zip(ws_third_ship.values, ws_third_ord.values)):
    if index == 0:
        ws_third["AAPL"] = [ws3_ship, ws3_ord]
    elif index == 1:
        ws_third["BBRD"] = [ws3_ship, ws3_ord]
    elif index == 2:
        ws_third["CCC"] = [ws3_ship, ws3_ord]

In [45]:
print(f"First day shipment and orders: {ws_first}")
print(f"Second day shipment and orders: {ws_second}")
print(f"Third day shipment and orders: {ws_third}")

First day shipment and orders: {'AAPL': [150, 320], 'BBRD': [700, 530], 'CCC': [450, 500]}
Second day shipment and orders: {'AAPL': [350, 330], 'BBRD': [600, 660], 'CCC': [350, 230]}
Third day shipment and orders: {'AAPL': [250, 340], 'BBRD': [800, 700], 'CCC': [550, 590]}


In [46]:
ws = {} #This dictionary will later contain the order and expected shipment quantities on the three dates
        #at wall street

ws["First Day"] = ws_first
ws["Second Day"] = ws_second
ws["Third Day"] = ws_third

In [47]:
ws

{'First Day': {'AAPL': [150, 320], 'BBRD': [700, 530], 'CCC': [450, 500]},
 'Second Day': {'AAPL': [350, 330], 'BBRD': [600, 660], 'CCC': [350, 230]},
 'Third Day': {'AAPL': [250, 340], 'BBRD': [800, 700], 'CCC': [550, 590]}}

## Inventory at the end of the three days - Wall Street
 - 22/08/2020
 - 23/08/2020
 - 24/08/2020
 
 At the beginning of the time period, we assumed each location had in their inventory 500 of each product. Also due to some logistic issues, there would not be any shipment of apple pies to any of the locations.

In [48]:
base_aapl = 500 #Inventory quantity at the beginning of the time period
base_bbrd = 500
base_ccc = 500
invent_day = {}
day1_ws = {}
day2_ws = {}
day3_ws = {}

for index, day in enumerate(ws.values()):    
    if index == 0: #First day - 22nd
        day1_ws["AAPL"] = base_aapl - day.get("AAPL")[1] #get the order quantity of appe pies 
        day1_ws["BBRD"] = base_bbrd + ord_ship_bal(day.get("BBRD"))#get the quantity of BBRD left after the first day
        day1_ws["CCC"] = base_ccc + ord_ship_bal(day.get("CCC"))#get the quantity of CCC left after the first day
        
        #The base quantities will then be updated to the quantity left after the first day
        base_aapl = day1_ws.get("AAPL")#Inventory quantities at the beginning of the second day
        base_bbrd = day1_ws["BBRD"]
        base_ccc = day1_ws["CCC"]
    
    if index == 1:#Second day - 23rd
        #Quantities left in the inventory at the end of the second day
        day2_ws["AAPL"] = base_aapl - day.get("AAPL")[1]
        day2_ws["BBRD"] = base_bbrd + ord_ship_bal(day.get("BBRD"))
        day2_ws["CCC"] = base_ccc + ord_ship_bal(day.get("CCC"))
        
        base_aapl = day2_ws.get("AAPL")#Inventory quantities at the beginning of the third day
        base_bbrd = day2_ws["BBRD"]
        base_ccc = day2_ws["CCC"]
    
    if index == 2:#Third day - 24th
        day3_ws["AAPL"] = base_aapl - day.get("AAPL")[1]
        day3_ws["BBRD"] = base_bbrd + ord_ship_bal(day.get("BBRD"))
        day3_ws["CCC"] = base_ccc + ord_ship_bal(day.get("CCC"))
    

In [49]:
print(day1_ws)#Order and shipment balance on the first day at wall street
print(day2_ws)#Order and shipment balance on the second day at wall street
print(day3_ws)#Order and shipment balance on the third day at wall street

{'AAPL': 180, 'BBRD': 670, 'CCC': 450}
{'AAPL': -150, 'BBRD': 610, 'CCC': 570}
{'AAPL': -490, 'BBRD': 710, 'CCC': 530}


In [50]:
ws_invent = {} 

ws_invent["22/08/2020 inventory"] = day1_ws
ws_invent["23/08/2020 inventory"] = day2_ws
ws_invent["24/08/2020 inventory"] = day3_ws

In [51]:
ws_invent

{'22/08/2020 inventory': {'AAPL': 180, 'BBRD': 670, 'CCC': 450},
 '23/08/2020 inventory': {'AAPL': -150, 'BBRD': 610, 'CCC': 570},
 '24/08/2020 inventory': {'AAPL': -490, 'BBRD': 710, 'CCC': 530}}

## Poughkeepsie Location
The quantity of each product left in the inventory of the outlet on Poughkeepsie on the 3 dates was calculated in this section.

In [52]:
pf_first = {}#The order and expected shipment(The shipment wasnt processed) quantities on the first day

for index, pf1_ord in enumerate(pf_first_ord.values):
    if index == 0:
        pf_first["AAPL"] = [0, pf1_ord]#Since there was no shipment to 3PUFF on the first day
    elif index == 1:
        pf_first["BBRD"] = [0, pf1_ord]
    elif index == 2:
        pf_first["CCC"] = [0, pf1_ord]
        
pf_second = {}#The order and expected shipment(The shipment wasnt processed) quantities on the second day

for index,(pf2_ship, pf2_ord) in enumerate(zip(pf_sec_ship.values, pf_sec_ord.values)):
    if index == 0:
        pf_second["AAPL"] = [pf2_ship, pf2_ord]
    elif index == 1:
        pf_second["BBRD"] = [pf2_ship, pf2_ord]
    elif index == 2:
        pf_second["CCC"] = [pf2_ship, pf2_ord]
        
pf_third = {} #The order and expected shipment(The shipment wasnt processed) quantities on the third day

for index,(pf3_ship, pf3_ord) in enumerate(zip(pf_third_ship.values, pf_third_ord.values)):
    if index == 0:
        pf_third["AAPL"] = [pf3_ship, pf3_ord]
    elif index == 1:
        pf_third["BBRD"] = [pf3_ship, pf3_ord]
    elif index == 2:
        pf_third["CCC"] = [pf3_ship, pf3_ord]

In [53]:
print(f"First day shipment and orders: {pf_first}")
print(f"Second day shipment and orders: {pf_second}")
print(f"Third day shipment and orders: {pf_third}")

First day shipment and orders: {'AAPL': [0, 420], 'BBRD': [0, 250], 'CCC': [0, 290]}
Second day shipment and orders: {'AAPL': [200, 410], 'BBRD': [400, 600], 'CCC': [700, 420]}
Third day shipment and orders: {'AAPL': [200, 170], 'BBRD': [400, 660], 'CCC': [800, 440]}


In [54]:
pf = {} #This dictionary will later contain the order and expected shipment quantities on the three dates
        #at jane street

pf["First Day"] = pf_first
pf["Second Day"] = pf_second
pf["Third Day"] = pf_third

pf

{'First Day': {'AAPL': [0, 420], 'BBRD': [0, 250], 'CCC': [0, 290]},
 'Second Day': {'AAPL': [200, 410], 'BBRD': [400, 600], 'CCC': [700, 420]},
 'Third Day': {'AAPL': [200, 170], 'BBRD': [400, 660], 'CCC': [800, 440]}}

## Inventory at the end of the three days - Poughkeepsie
 - 22/08/2020
 - 23/08/2020
 - 24/08/2020
 
 At the beginning of the time period, we assumed each location had in their inventory 500 of each product. Also due to some logistic issues, there would not be any shipment of apple pies to any of the locations.

In [55]:
base_aapl = 500 #Inventory quantity at the beginning of the time period
base_bbrd = 500
base_ccc = 500
invent_day = {}
day1_pf = {}
day2_pf = {}
day3_pf = {}

for index, day in enumerate(pf.values()):    
    if index == 0: #First day - 22nd
        day1_pf["AAPL"] = base_aapl - day.get("AAPL")[1] #get the order quantity of appe pies 
        day1_pf["BBRD"] = base_bbrd + ord_ship_bal(day.get("BBRD"))#get the quantity of BBRD left after the first day
        day1_pf["CCC"] = base_ccc + ord_ship_bal(day.get("CCC"))#get the quantity of CCC left after the first day
        
        #The base quantities will then be updated to the quantity left after the first day
        base_aapl = day1_pf.get("AAPL")#Inventory quantities at the beginning of the second day
        base_bbrd = day1_pf["BBRD"]
        base_ccc = day1_pf["CCC"]
    
    if index == 1:#Second day - 23rd
        #Quantities left in the inventory at the end of the second day
        day2_pf["AAPL"] = base_aapl - day.get("AAPL")[1]
        day2_pf["BBRD"] = base_bbrd + ord_ship_bal(day.get("BBRD"))
        day2_pf["CCC"] = base_ccc + ord_ship_bal(day.get("CCC"))
        
        base_aapl = day2_pf.get("AAPL")#Inventory quantities at the beginning of the third day
        base_bbrd = day2_pf["BBRD"]
        base_ccc = day2_pf["CCC"]
    
    if index == 2:#Third day - 24th
        day3_pf["AAPL"] = base_aapl - day.get("AAPL")[1]
        day3_pf["BBRD"] = base_bbrd + ord_ship_bal(day.get("BBRD"))
        day3_pf["CCC"] = base_ccc + ord_ship_bal(day.get("CCC"))
    

In [56]:
print(day1_pf)
print(day2_pf)
print(day3_pf)

{'AAPL': 80, 'BBRD': 250, 'CCC': 210}
{'AAPL': -330, 'BBRD': 50, 'CCC': 490}
{'AAPL': -500, 'BBRD': -210, 'CCC': 850}


In [57]:
pf_invent = {} 

pf_invent["22/08/2020 inventory"] = day1_pf
pf_invent["23/08/2020 inventory"] = day2_pf
pf_invent["24/08/2020 inventory"] = day3_pf

In [58]:
pf_invent

{'22/08/2020 inventory': {'AAPL': 80, 'BBRD': 250, 'CCC': 210},
 '23/08/2020 inventory': {'AAPL': -330, 'BBRD': 50, 'CCC': 490},
 '24/08/2020 inventory': {'AAPL': -500, 'BBRD': -210, 'CCC': 850}}

In [59]:
inventory ={}

inventory["1JS"]=js_invent
inventory["2WS"]=ws_invent
inventory["3PUFF"]=pf_invent

In [60]:
inventory #inventory for the three dates at the three different locations

{'1JS': {'22/08/2020 inventory': {'AAPL': -180, 'BBRD': 880, 'CCC': 510},
  '23/08/2020 inventory': {'AAPL': -700, 'BBRD': 690, 'CCC': -750},
  '24/08/2020 inventory': {'AAPL': -1470, 'BBRD': 180, 'CCC': -1390}},
 '2WS': {'22/08/2020 inventory': {'AAPL': 180, 'BBRD': 670, 'CCC': 450},
  '23/08/2020 inventory': {'AAPL': -150, 'BBRD': 610, 'CCC': 570},
  '24/08/2020 inventory': {'AAPL': -490, 'BBRD': 710, 'CCC': 530}},
 '3PUFF': {'22/08/2020 inventory': {'AAPL': 80, 'BBRD': 250, 'CCC': 210},
  '23/08/2020 inventory': {'AAPL': -330, 'BBRD': 50, 'CCC': 490},
  '24/08/2020 inventory': {'AAPL': -500, 'BBRD': -210, 'CCC': 850}}}

In [61]:
js_df = pd.DataFrame(inventory["1JS"].values(), index = ["22/08/2020","23/08/2020","24/08/2020"])
js_df#DataFrame showing the 1JS inventory balance of each item at each date

,AAPL,BBRD,CCC
22/08/2020,-180,880,510
23/08/2020,-700,690,-750
24/08/2020,-1470,180,-1390


In [62]:
ws_df = pd.DataFrame(inventory["2WS"].values(), index = ["22/08/2020","23/08/2020","24/08/2020"])
ws_df#DataFrame showing the 2WS inventory balance of each item at each date

,AAPL,BBRD,CCC
22/08/2020,180,670,450
23/08/2020,-150,610,570
24/08/2020,-490,710,530


In [63]:
pf_df = pd.DataFrame(inventory["3PUFF"].values(), index = ["22/08/2020","23/08/2020","24/08/2020"])
pf_df#DataFrame showing the 3PUFF inventory balance of each item at each date

,AAPL,BBRD,CCC
22/08/2020,80,250,210
23/08/2020,-330,50,490
24/08/2020,-500,-210,850


## Shipment deficit/surplus at the mega bakery
The daily production capacity of the megabakery:
 - 2000 BBRD
 - 1600 CC
 - 700 AAPL
 
 Since the products leave the megabakery/shipped from the megabakery on the activity dates, the activity dates will be taken into  consideration to know the quantity of products leaving the megabakery each day.

In [64]:
dates = shipment_df.groupby("Activity Date")

In [65]:
shipment_df["Activity Date"].value_counts() #get dates products were shipped from the megabakery

23/08/2020    9
21/08/2020    9
22/08/2020    9
Name: Activity Date, dtype: int64

In [66]:
first_ship_date = dates.get_group("21/08/2020")

In [67]:
first_ship_date.groupby("Bakery Item").sum()[["Quantity"]]

,Quantity
Bakery Item,
AAPL,700
BBRD,2000
CCC,1600


In [68]:
sec_ship_date = dates.get_group("22/08/2020")

In [69]:
sec_ship_date.groupby("Bakery Item").sum()[["Quantity"]]

,Quantity
Bakery Item,
AAPL,700
BBRD,2000
CCC,1600


In [70]:
third_ship_date = dates.get_group("23/08/2020")

In [71]:
third_ship_date.groupby("Bakery Item").sum()[["Quantity"]]

,Quantity
Bakery Item,
AAPL,700
BBRD,2000
CCC,1600


It can be seen from the above that the mega bakery worked at maximum capacity every day during the period.

In [72]:
js_df

,AAPL,BBRD,CCC
22/08/2020,-180,880,510
23/08/2020,-700,690,-750
24/08/2020,-1470,180,-1390


In [73]:
ws_df

,AAPL,BBRD,CCC
22/08/2020,180,670,450
23/08/2020,-150,610,570
24/08/2020,-490,710,530


In [74]:
pf_df

,AAPL,BBRD,CCC
22/08/2020,80,250,210
23/08/2020,-330,50,490
24/08/2020,-500,-210,850


## What if there was no issue with the transport of Apple pies to 1JS?

In [75]:
base_aapl = 500 #Inventory quantity at the beginning of the time period
base_bbrd = 500
base_ccc = 500
invent_day = {}
day1_js = {}
day2_js = {}
day3_js = {}

for index, day in enumerate(js.values()):    
    if index == 0: #First day - 22nd
        day1_js["AAPL"] = base_aapl + ord_ship_bal(day.get("AAPL")) #get the order quantity of appe pies 
        day1_js["BBRD"] = base_bbrd + ord_ship_bal(day.get("BBRD"))#get the quantity of BBRD left after the first day
        day1_js["CCC"] = base_ccc + ord_ship_bal(day.get("CCC"))#get the quantity of CCC left after the first day
        
        #The base quantities will then be updated to the quantity left after the first day
        base_aapl = day1_js.get("AAPL")#Inventory quantities at the beginning of the second day
        base_bbrd = day1_js["BBRD"]
        base_ccc = day1_js["CCC"]
    
    if index == 1:#Second day - 23rd
        #Quantities left in the inventory at the end of the second day
        day2_js["AAPL"] = base_aapl + ord_ship_bal(day.get("AAPL"))
        day2_js["BBRD"] = base_bbrd + ord_ship_bal(day.get("BBRD"))
        day2_js["CCC"] = base_ccc + ord_ship_bal(day.get("CCC"))
        
        base_aapl = day2_js.get("AAPL")#Inventory quantities at the beginning of the third day
        base_bbrd = day2_js["BBRD"]
        base_ccc = day2_js["CCC"]
    
    if index == 2:#Third day - 24th
        day3_js["AAPL"] = base_aapl + ord_ship_bal(day.get("AAPL"))
        day3_js["BBRD"] = base_bbrd + ord_ship_bal(day.get("BBRD"))#get the quantity of BBRD left after the first day
        day3_js["CCC"] = base_ccc + ord_ship_bal(day.get("CCC"))#get the quantity of CCC left after the first day

In [76]:
js_invent = {}

js_invent["22/08/2020 inventory"] = day1_js
js_invent["23/08/2020 inventory"] = day2_js
js_invent["24/08/2020 inventory"] = day3_js

js_invent

{'22/08/2020 inventory': {'AAPL': 170, 'BBRD': 880, 'CCC': 510},
 '23/08/2020 inventory': {'AAPL': -200, 'BBRD': 690, 'CCC': -750},
 '24/08/2020 inventory': {'AAPL': -670, 'BBRD': 180, 'CCC': -1390}}

In [77]:
js_df = pd.DataFrame(js_invent.values(), index = ["22/08/2020","23/08/2020","24/08/2020"])
js_df#DataFrame showing the 1JS inventory balance if there was no issue with apple pie transport

,AAPL,BBRD,CCC
22/08/2020,170,880,510
23/08/2020,-200,690,-750
24/08/2020,-670,180,-1390


The deficits in Apple pies would have been minimized if the issue with the shipment of Apple pies was fixed.

## What if there was no issue with the transport of Apple pies to 2WS?

In [78]:
base_aapl = 500 #Inventory quantity at the beginning of the time period
base_bbrd = 500
base_ccc = 500
invent_day = {}
day1_ws = {}
day2_ws = {}
day3_ws = {}

for index, day in enumerate(ws.values()):    
    if index == 0: #First day - 22nd
        day1_ws["AAPL"] = base_aapl + ord_ship_bal(day.get("AAPL")) #get the order quantity of appe pies 
        day1_ws["BBRD"] = base_bbrd + ord_ship_bal(day.get("BBRD"))#get the quantity of BBRD left after the first day
        day1_ws["CCC"] = base_ccc + ord_ship_bal(day.get("CCC"))#get the quantity of CCC left after the first day
        
        #The base quantities will then be updated to the quantity left after the first day
        base_aapl = day1_ws.get("AAPL")#Inventory quantities at the beginning of the second day
        base_bbrd = day1_ws["BBRD"]
        base_ccc = day1_ws["CCC"]
    
    if index == 1:#Second day - 23rd
        #Quantities left in the inventory at the end of the second day
        day2_ws["AAPL"] = base_aapl + ord_ship_bal(day.get("AAPL"))
        day2_ws["BBRD"] = base_bbrd + ord_ship_bal(day.get("BBRD"))
        day2_ws["CCC"] = base_ccc + ord_ship_bal(day.get("CCC"))
        
        base_aapl = day2_ws.get("AAPL")#Inventory quantities at the beginning of the third day
        base_bbrd = day2_ws["BBRD"]
        base_ccc = day2_ws["CCC"]
    
    if index == 2:#Third day - 24th
        day3_ws["AAPL"] = base_aapl + ord_ship_bal(day.get("AAPL"))
        day3_ws["BBRD"] = base_bbrd + ord_ship_bal(day.get("BBRD"))
        day3_ws["CCC"] = base_ccc + ord_ship_bal(day.get("CCC"))
    

In [79]:
ws_invent = {} 

ws_invent["22/08/2020 inventory"] = day1_ws
ws_invent["23/08/2020 inventory"] = day2_ws
ws_invent["24/08/2020 inventory"] = day3_ws

In [80]:
ws_invent

{'22/08/2020 inventory': {'AAPL': 330, 'BBRD': 670, 'CCC': 450},
 '23/08/2020 inventory': {'AAPL': 350, 'BBRD': 610, 'CCC': 570},
 '24/08/2020 inventory': {'AAPL': 260, 'BBRD': 710, 'CCC': 530}}

In [81]:
ws_df = pd.DataFrame(ws_invent.values(), index = ["22/08/2020","23/08/2020","24/08/2020"])
ws_df#DataFrame showing the 2WS inventory balance if there was no issue with apple pie transport

,AAPL,BBRD,CCC
22/08/2020,330,670,450
23/08/2020,350,610,570
24/08/2020,260,710,530


As seen above, the apple pie deficits would have been solved if there was no issue with the transport of apple pies

## What if shipment issues to Poughkeepsie were fixed?

In [82]:
base_aapl = 500 #Inventory quantity at the beginning of the time period
base_bbrd = 500
base_ccc = 500
invent_day = {}
day1_pf = {}
day2_pf = {}
day3_pf = {}

for index, day in enumerate(pf.values()):    
    if index == 0: #First day - 22nd
        day1_pf["AAPL"] = base_aapl + ord_ship_bal(day.get("AAPL")) #get the order quantity of appe pies 
        day1_pf["BBRD"] = base_bbrd + ord_ship_bal(day.get("BBRD"))#get the quantity of BBRD left after the first day
        day1_pf["CCC"] = base_ccc + ord_ship_bal(day.get("CCC"))#get the quantity of CCC left after the first day
        
        #The base quantities will then be updated to the quantity left after the first day
        base_aapl = day1_pf.get("AAPL")#Inventory quantities at the beginning of the second day
        base_bbrd = day1_pf["BBRD"]
        base_ccc = day1_pf["CCC"]
    
    if index == 1:#Second day - 23rd
        #Quantities left in the inventory at the end of the second day
        day2_pf["AAPL"] = base_aapl + ord_ship_bal(day.get("AAPL"))
        day2_pf["BBRD"] = base_bbrd + ord_ship_bal(day.get("BBRD"))
        day2_pf["CCC"] = base_ccc + ord_ship_bal(day.get("CCC"))
        
        base_aapl = day2_pf.get("AAPL")#Inventory quantities at the beginning of the third day
        base_bbrd = day2_pf["BBRD"]
        base_ccc = day2_pf["CCC"]
    
    if index == 2:#Third day - 24th
        day3_pf["AAPL"] = base_aapl + ord_ship_bal(day.get("AAPL"))
        day3_pf["BBRD"] = base_bbrd + ord_ship_bal(day.get("BBRD"))
        day3_pf["CCC"] = base_ccc + ord_ship_bal(day.get("CCC"))
    

In [83]:
pf_invent = {} 

pf_invent["22/08/2020 inventory"] = day1_pf
pf_invent["23/08/2020 inventory"] = day2_pf
pf_invent["24/08/2020 inventory"] = day3_pf

In [84]:
pf_df = pd.DataFrame(pf_invent.values(), index = ["22/08/2020","23/08/2020","24/08/2020"])
pf_df#DataFrame showing the 3PUFF inventory balance if there was no issue with apple pie transport

,AAPL,BBRD,CCC
22/08/2020,80,250,210
23/08/2020,-130,50,490
24/08/2020,-100,-210,850


In [85]:
df.to_excel("excel_notebook_alalade.xlsx", index=False)